In [ ]:
# HEAVY MODEL
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Define the saliency detection model (example using a pre-trained ResNet)
class SaliencyModel(nn.Module):
    def __init__(self):
        super(SaliencyModel, self).__init__()
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()  # Remove the final fully connected layer
        self.conv = nn.Conv2d(2048, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.conv(x)
        x = self.sigmoid(x)
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = SaliencyModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)

        # Create dummy saliency labels
        labels = torch.ones_like(outputs)  # All pixels are salient (dummy saliency map)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


100%|██████████| 170498071/170498071 [00:01<00:00, 98567381.16it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 62.9MB/s]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Define the saliency detection model (example using a pre-trained MobileNetV2)
class SaliencyModel(nn.Module):
    def __init__(self):
        super(SaliencyModel, self).__init__()
        self.backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).features
        self.conv = nn.Conv2d(1280, 1, kernel_size=1)  # Adjust the input channels based on the backbone
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = self.conv(x)
        x = self.sigmoid(x)
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduced image size
    transforms.ToTensor()
])

# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = SaliencyModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)

        # Create dummy saliency labels
        labels = torch.ones_like(outputs)  # All pixels are salient (dummy saliency map)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Check for CUDA availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Define the saliency detection model (example using a pre-trained MobileNetV2)
class SaliencyModel(nn.Module):
    def __init__(self):
        super(SaliencyModel, self).__init__()
        self.backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).features
        self.conv = nn.Conv2d(1280, 1, kernel_size=1)  # Adjust the input channels based on the backbone

    def forward(self, x):
        x = self.backbone(x)
        x = self.conv(x)
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduced image size
    transforms.ToTensor()
])

# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = SaliencyModel().to(device)
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for numerical stability
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Training loop
for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)

            # Create dummy saliency labels
            labels = torch.ones_like(outputs)  # All pixels are salient (dummy saliency map)
            labels = labels.to(device)

            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Using device: cuda
Files already downloaded and verified
Epoch 1, Loss: 9.39589699555654e-06
Epoch 2, Loss: 2.270758614031365e-06
Epoch 3, Loss: 8.063663017310319e-07
Epoch 4, Loss: 3.219837481083232e-07
Epoch 5, Loss: 1.3732977777181077e-07
Epoch 6, Loss: 6.214696668394026e-08
Epoch 7, Loss: 4.484113702574177e-08
Epoch 8, Loss: 4.228511940596036e-08
Epoch 9, Loss: 3.765539702271781e-08
Epoch 10, Loss: 3.4411190341643305e-08


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.segmentation import deeplabv3_resnet50

# Check for CUDA availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Define the interactive segmentation model (example using DeepLabV3)
class InteractiveSegmentationModel(nn.Module):
    def __init__(self):
        super(InteractiveSegmentationModel, self).__init__()
        self.backbone = deeplabv3_resnet50(weights='DEFAULT')

        # Modify the first convolution layer to accept 4 channels
        self.backbone.backbone.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x, clicks):
        # Incorporate clicks into the input (this is a simplified example)
        x = torch.cat((x, clicks), dim=1)
        x = self.backbone(x)['out']
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Download CIFAR-10 dataset as an example
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = InteractiveSegmentationModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Training loop
for epoch in range(10):
    model.train()
    for images, _ in train_loader:
        images = images.to(device)

        # Replace with actual click data
        clicks = torch.zeros((images.size(0), 1, images.size(2), images.size(3)), device=device)

        # Create dummy segmentation labels
        labels = torch.randint(0, 2, (images.size(0), images.size(2), images.size(3)), dtype=torch.long, device=device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images, clicks)

            # Resize labels to match the output size
            labels = torch.nn.functional.interpolate(labels.unsqueeze(1).float(), size=outputs.shape[2:], mode='nearest').squeeze(1).long()

            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Using device: cuda
Files already downloaded and verified
Epoch 1, Loss: 0.6934249401092529
Epoch 2, Loss: 0.6934133768081665
Epoch 3, Loss: 0.6934886574745178
Epoch 4, Loss: 0.6933601498603821
Epoch 5, Loss: 0.6933375000953674
Epoch 6, Loss: 0.6933647990226746
Epoch 7, Loss: 0.6934226155281067
Epoch 8, Loss: 0.6933717727661133
Epoch 9, Loss: 0.6933818459510803
Epoch 10, Loss: 0.6934298872947693


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.segmentation import deeplabv3_resnet50

# Define the interactive segmentation model (example using DeepLabV3)
class InteractiveSegmentationModel(nn.Module):
    def __init__(self):
        super(InteractiveSegmentationModel, self).__init__()
        self.backbone = deeplabv3_resnet50(weights='DEFAULT')

    def forward(self, x, clicks):
        # Incorporate clicks into the input (this is a simplified example)
        x = torch.cat((x, clicks), dim=1)
        x = self.backbone(x)['out']
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder('/path/to/train/data', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = InteractiveSegmentationModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        # Replace with actual click data in practice
        clicks = torch.zeros((images.size(0), 1, images.size(2), images.size(3)), device=images.device)

        optimizer.zero_grad()
        outputs = model(images, clicks)

        # Resize labels to match the output size
        labels = torch.nn.functional.interpolate(labels.unsqueeze(1).float(), size=outputs.shape[2:], mode='nearest').squeeze(1).long()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


In [ ]:
def run_pipeline(image, user_clicks):
    # Stage 1: Salient Object Detection
    saliency_map = saliency_model(image)

    # Stage 2: Interactive Segmentation
    refined_segmentation = InteractiveSegmentationModel(image, user_clicks)

    return refined_segmentation

# Example usage
image = load_image('/path/to/image.jpg')
user_clicks = get_user_clicks()  # This function should capture user inputs
final_segmentation = run_pipeline(image, user_clicks)
save_segmentation(final_segmentation, '/path/to/save/segmentation.png')


In [ ]:
# Use an official PyTorch runtime as a parent image
FROM pytorch/pytorch:latest

# Set the working directory
WORKDIR /app

# Copy the current directory contents into the container
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Run the application
CMD ["python", "inference_script.py"]


In [ ]:
import torch
from unet_model import UNet
from interactive_segmentation_model import InteractiveSegmentation
from PIL import Image
import numpy as np

# Load models
saliency_model = UNet(n_channels=3, n_classes=1)
interactive_model = InteractiveSegmentation(n_channels=3, n_classes=1)

saliency_model.load_state_dict(torch.load('saliency_model.pth'))
interactive_model.load_state_dict(torch.load('interactive_model.pth'))

saliency_model.eval()
interactive_model.eval()

# Inference function
def segment_image(image_path, clicks):
    image = Image.open(image_path)
    image = image.resize((256, 256))
    image = np.array(image).transpose((2, 0, 1)) / 255.0  # Normalize and convert to tensor
    image = torch.Tensor(image).unsqueeze(0)

    with torch.no_grad():
        saliency_output = saliency_model(image)
        refined_output = interactive_model(image, clicks)

    return refined_output

# Example usage
clicks = torch.Tensor([[[100, 100], [150, 150], [100, 150], [150, 100]]])  # Example clicks
segmented_image = segment_image('example_image.jpg', clicks)
